In [ ]:
from cyschoolhousesuite import *
from simple_cysh import *
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import numpy as np

In [ ]:
def click_delete(driver):
    driver.find_element_by_xpath('//input[contains(@value, "Delete")]').click()
    EC.alert_is_present
    driver.switch_to.alert.accept()
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/table/tbody/tr/td[2]/div[2]/form/div/span/span[1]/input')))

sch_ref_path = "Z:\\ChiPrivate\\Chicago Data and Evaluation\\SY18\\SY18 Team Placement\\SY18 FINAL Team Placements.xlsx"
sch_ref_df = pd.read_excel(sch_ref_path, sheet_name="School Data")

d = {'Reporting Perdiod': ['Quarter 1', 'Quarter 2', 'Quarter 3', 'Quarter 4'], 
     'Qtr': ['Qtr 1', 'Qtr 2', 'Qtr 3', 'Qtr 4'],
     'Q': ['Q1', 'Q2', 'Q3', 'Q4'],
     'Due Date': ['2017-11-02', '2018-02-01', '2018-04-12', '2018-06-18'],
     'Time__c': ['a1Y1a000000GlR5EAK', 'a1Y1a000000GlQNEA0', 'a1Y1a000000GlQOEA0', 'a1Y1a000000GlQPEA0']}

gd_assign = pd.DataFrame(d)
subjects = ['Math', 'ELA']

In [ ]:
section_df = get_cysh_df('Section__c', ['Id','Name', 'Student_Section_Active_Count__c'], rename_id=True)
gb_sections_df = section_df.loc[section_df['Name'].str.contains("Grade ") & (section_df['Student_Section_Active_Count__c']>0)].copy()
gb_sections_df.loc[gb_sections_df.Name.str.contains('ELA'), 'Subject'] = 'ELA'
gb_sections_df.loc[gb_sections_df.Name.str.contains('Math'), 'Subject'] = 'Math'

assgnmt_df = get_cysh_df('Assignment__c', ['Id','Name', 'IsDeleted', 'Section__c', 'Assignment_Library__c','Due_Date__c', 'Due_Date_for_Gradebook__c', 'Color__c', 'Name_in_Gradebook__c', 'Picklist_Value__c', 'Time__c', 'Weighting_Value__c'], rename_id=True)
assgnmt_df = assgnmt_df.loc[assgnmt_df['IsDeleted']==False]
assgnmt_lib_df = get_cysh_df('Assignment_Lib__c', ['Id','Name'], rename_id=True, rename_name=True)
assgnmt_df = assgnmt_df.merge(assgnmt_lib_df, left_on='Assignment_Library__c', right_on='Assignment_Lib__c', how='left')

# Generate Master Record of Expected Assignments
assignment_list = []
for quarter in ['Qtr 1', 'Qtr 2', 'Qtr 3', 'Qtr 4']:
    for subject in ['ELA', 'Math']:
        assignment_list.append([f'{quarter} - {subject} - Reporting Period Course Grade - Site', subject, quarter])
assgnmts = pd.DataFrame(assignment_list, columns=['Assignment', 'Subject', 'Quarter'])
expct_assgnmts = gb_sections_df[['Section__c', 'Name', 'Subject']].merge(assgnmts, on='Subject')
expct_assgnmts['Key'] = expct_assgnmts['Name'] + '_' + expct_assgnmts['Assignment']

gb_sections_df = gb_sections_df.merge(assgnmt_df, how='left', on='Section__c', suffixes=['_sect', '_asgn'])
gb_sections_df['Key'] = gb_sections_df['Name_sect'] + '_' + gb_sections_df['Assignment_Lib__c_Name']

assgmts_to_make = expct_assgnmts.loc[~expct_assgnmts['Key'].isin(gb_sections_df['Key'])]

In [ ]:
assgmts_to_make

In [ ]:
counts = gb_sections_df.groupby(['Section__c', 'Name_sect'])['Name_asgn'].count().reset_index()
counts = counts.loc[counts['Name_asgn']!=4]
counts

In [ ]:
for i, r in  assgmts_to_make.iterrows():
    asgnmt_lib__c = assgnmt_lib_df[assgnmt_lib_df['Assignment_Lib__c_Name']==f"{r['Quarter']} - {r['Subject']} - Reporting Period Course Grade - Site"]['Assignment_Lib__c'].values[0]

    query_dict = {'Name':f"{r['Quarter']} - {r['Subject']} - Reporting Period Course Grade - CHI",
                  'Section__c': r['Section__c'],
                  'Assignment_Library__c': asgnmt_lib__c,
                  'Due_Date__c':gd_assign.loc[gd_assign['Qtr']==r['Quarter'], 'Due Date'].values[0],
                  'Due_Date_for_Gradebook__c':gd_assign.loc[gd_assign['Qtr']==r['Quarter'], 'Due Date'].values[0],
                  'Picklist_Value__c': 'a0r1a000001mjEFAAY', # Grading Scale
                  'Time__c': gd_assign.loc[gd_assign['Qtr']==r['Quarter'], 'Time__c'].values[0], # Reporting Period
                  'Weighting_Value__c': 1.0}
    if r['Subject']=='Math':
        query_dict['Color__c'] = 'green'
        query_dict['Name_in_Gradebook__c'] = gd_assign.loc[gd_assign['Qtr']==r['Quarter'], 'Q'].values[0] + ' MA'
    elif r['Subject']=='ELA':
        query_dict['Color__c'] = 'blue'
        query_dict['Name_in_Gradebook__c'] = gd_assign.loc[gd_assign['Qtr']==r['Quarter'], 'Q'].values[0] + 'ELA'

    print(query_dict)
    print("")
            
#     cysh.Assignment__c.create(query_dict)

# SalesforceMalformedRequest: 'Unable to create/update fields':
#     Name_in_Gradebook__c
#     Time__c
#     Due_Date_for_Gradebook__c
#     Picklist_Value__c
#     Weighting_Value__c

# gb_sections_df[gb_sections_df['Name'].str.contains("Chalmers")]

In [ ]:
assign_df = get_report('00O1a000002rwMN')
assign_df['key'] = assign_df['Assignment: Section: Section Name'] + assign_df['Reporting Period Name']

#assign_df = assign_df[assign_df['Reporting Period Name']='Quarter 1']
#assign_df.drop_duplicates('Assignment: Assignment ID', inplace=True)
#assign_df = assign_df[assign_df.duplicated('key', keep='first')]

assign_counts = assign_df.groupby(by=['key', 'Assignment: Assignment ID']).count()
assign_counts[assign_counts['Entered Grade']==0]

In [ ]:
# Add Students to Gradebook Sections
driver = get_driver()
open_cyschoolhouse18(driver)

wait = 10

for school in sch_ref_df['School'].tolist():
    print("Starting: {}".format(school))
    driver.get("https://c.na24.visual.force.com/apex/core_ChangeSchool")
    driver.find_element_by_xpath("//select[@name='j_id0:j_id30:j_id32']/option[text()=\"%s\"]" % school).click()
    sleep(3)
    driver.find_element_by_xpath("//input[contains(@class, 'btn') and contains(@value, 'Change')]").click()
    sleep(3)
    for i, r in gd_assign.ix[1:].iterrows():
        for x in subjects:
            driver.get('https://c.na24.visual.force.com/apex/gradebook_addassignments_v2')
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//li[contains(@id, 'addExistingTab')]")))
            driver.find_element_by_xpath("//li[contains(@id, 'addExistingTab')]").click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//select[contains(@name, 'j_id0:mainForm:j_id75:j_id88')]")))
            driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id88']/option[text()='Reporting Period Course Grade']").click()
            # wait for box to appear/disapper
            fancy_box_wait(waittime=10)
            driver.find_element_by_xpath("//input[@name='j_id0:mainForm:j_id75:j_id84']").send_keys(r['Qtr'])
            driver.find_element_by_xpath("//input[contains(@class, 'btn drk_blue_btn') and contains(@value, 'Search Assignments')]").click()
            fancy_box_wait(waittime=10)
            
            if x=='Math':
                # choose math, enter credentials
                driver.find_element_by_xpath("//td[contains(text(), \"%s\")]" % (r['Qtr'] + " - " + x)).click()
                fancy_box_wait(waittime=10)
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").clear()
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").send_keys(r['Qtr'] + " - " + x + " - Reporting Period Course Grade - Chicago")
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id166')]").send_keys(r['Q'] + ' Ma')
                # choose blue
                driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id211']/option[text()='green']").click()
            
            elif x=='ELA':
                # choose ELA, enter credentials
                driver.find_element_by_xpath("//td[contains(text(), \"%s\")]" % (r['Qtr'] + " - " + x)).click()
                fancy_box_wait(waittime=10)
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").clear()
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id153')]").send_keys(r['Qtr'] + " - " + x + " - Reporting Period Course Grade - Chicago")
                driver.find_element_by_xpath("//input[contains(@name, 'j_id0:mainForm:j_id75:j_id166')]").send_keys(r['Q'] + 'ELA')
                # choose blue
                driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id211']/option[text()='blue']").click()
            
            driver.find_element_by_xpath("//select[@name='j_id0:mainForm:j_id75:j_id232']/option[text()='A-F']").click()
            fancy_box_wait(waittime=10)
            # select all
            driver.find_element_by_xpath("//input[contains(@onclick, \"%s\")]" % "selectAll2(this,'addAssCb');").click()

            for elem in driver.find_elements_by_xpath("//select[contains(@class, 'rPeriod')]"):
                ID=elem.get_attribute('id')
                driver.find_element_by_xpath("//select[@id='{}']/option[text()='{}']".format(ID, r['Reporting Perdiod'])).click()
    
            for elem in driver.find_elements_by_xpath("//input[contains(@class, 'datepicker hasDatepicker')]"):
                elem.clear()
                elem.send_keys(r['Due Date'])
           
            driver.find_element_by_xpath("//input[contains(@class, 'btn black_btn') and contains(@value, 'Submit')]").click()
            
            fancy_box_wait(waittime=10)